***Import the Pandas module***

In [ ]:
import numpy as np
import pandas as pd

## **Aggregation in Pandas**

In [ ]:
ran = np.random.RandomState(45)
ser = pd.Series(ran.rand(7))
ser

0    0.989012
1    0.549545
2    0.281447
3    0.077290
4    0.444469
5    0.472808
6    0.048522
dtype: float64

In [ ]:
print('the sum of the series values is:',ser.sum())
print('the mean of the series is:',ser.mean())

the sum of the series values is: 2.863092574988295
the mean of the series is: 0.4090132249983279


the dataframe by default **aggregate function** return in column types

In [ ]:
d = pd.DataFrame({'A': ran.rand(7),
                   'B': ran.rand(7)})
d

,A,B
0,0.163324,0.618294
1,0.115951,0.282667
2,0.627392,0.976003
3,0.856182,0.673068
4,0.650102,0.440531
5,0.990722,0.289687
6,0.470351,0.509700


the **mean** of the rows and column

In [ ]:
print('the mean of columns  is :\n',d.mean())
print('the meanof rows is :\n',d.mean(axis='columns'))

the mean of columns  is :
 A    0.553432
B    0.541422
dtype: float64
the meanof rows is :
 0    0.390809
1    0.199309
2    0.801697
3    0.764625
4    0.545317
5    0.640205
6    0.490025
dtype: float64


In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


***GroupBy:*** *Split, Apply, Combine: Simple aggregations can give you a flavor of your dataset, but often we would prefer to aggregate conditionally on some label or index: this is implemented in the so-called groupby operation.*

In [ ]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
print(df)

  key  data
0   A     0
1   B     1
2   C     2
3   A     3
4   B     4
5   C     5


*The most basic split-apply-combine operation can be computed with the groupby() method of DataFrames, passing the name of the desired key column:*

In [ ]:
df.groupby('key')

*To produce a result, we can apply an aggregate to this DataFrameGroupBy object, which will perform the appropriate apply/combine steps to produce the desired result:*

In [ ]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


*The GroupBy object: The GroupBy object is a very flexible abstraction. In many ways, you can simply treat it as if it's a collection of DataFrames, and it does the difficult things under the hood.*

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
print(df)

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9


In [ ]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

In [ ]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


***Filtering:***

*Filtering A filtering operation allows you to drop data based on the group properties. For example, we might want to keep all groups in which the standard deviation is larger than some critical value:*

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
print(df)

  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9


In [ ]:
def filter_func(x):
    return x['data2'].std() > 4

In [ ]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [ ]:
df.groupby('key').std()

,data1,data2
key,,
A,2.12132,1.414214
B,2.12132,4.949747
C,2.12132,4.242641


In [ ]:
df.groupby('key').filter(filter_func)

,key,data1,data2
1,B,1,0
2,C,2,3
4,B,4,7
5,C,5,9


*The filter function should return a Boolean value specifying whether the group passes the filtering. Here because group A does not have a standard deviation greater than 4, it is dropped from the result.*

***Transformation:***

*Transformation While aggregation must return a reduced version of the data, transformation can return some transformed version of the full data to recombine. For such a transformation, the output is the same shape as the input. A common example is to center the data by subtracting the group-wise mean:*

In [ ]:
df.groupby('key').transform(lambda x:x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


***Apply Method():***

*The apply() method The apply() method lets you apply an arbitrary function to the group results. The function should take a DataFrame, and return either a Pandas object (e.g., DataFrame, Series) or a scalar; the combine operation will be tailored to the type of output returned.*

*For example, here is an apply() that normalizes the first column by the sum of the second:*

In [ ]:
def norm_by_data2(x):
    x['data1'] /= x['data2'].sum()
    return x

In [ ]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [ ]:
df.groupby('key').apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9
